### Imports

In [1]:
import pandas as pd
import numpy as np
import statsbombpy as sb

In [2]:
passesDF= pd.read_parquet('../Data/passes.parquet')
carriesDF= pd.read_parquet('../Data/carrys.parquet')
itcDF= pd.read_parquet('../Data/inteceptions.parquet')
dribblesDF= pd.read_parquet('../Data/dribbles.parquet')

## Players aggregated stats  
### Passes

1. Progressive Passes

In [3]:
# Progressive passes - passes made under pressure - completed passes made under pressure - passes under pressure comletion rate
# Goal coordinates    
goal_X= 120
goal_Y= 40

# Add starting location coordinates
passesDF[['X', 'Y']] = pd.DataFrame(passesDF['location'].to_list(), index= passesDF.index)

# Add end location coordinates
passesDF[['end_X', 'end_Y']]= pd.DataFrame(passesDF['pass_end_location'].to_list(), index= passesDF.index)

# add the distance between starting point and the goal (progressie passes calculation):
passesDF['start_distance']= np.sqrt((goal_X - passesDF.X)**2 + (goal_Y - passesDF.Y)**2)
passesDF['end_distance']= np.sqrt((goal_X - passesDF.end_X)**2 + (goal_Y - passesDF.end_Y)**2)

# check if a pass is progressive or not
passesDF['isProgressive'] = ((abs(passesDF['start_distance'] - passesDF['end_distance']) >= 0.25 * passesDF['start_distance']) & 
                             (abs(passesDF['Y'] - passesDF['end_Y']) >= 5)).tolist()

progressive_passes_DF = passesDF.groupby(["player_id", "player"])["isProgressive"].sum().reset_index()
progressive_passes_DF.rename(columns= {'isProgressive':'progressive_passes'}, inplace= True )
progressive_passes_DF

,player_id,player,progressive_passes
0,2941,Ismaïla Sarr,22
1,2954,Youri Tielemans,10
2,2972,Marcus Thuram,4
3,2988,Memphis Depay,23
4,2989,Alexander Djiku,4
...,...,...,...
671,184468,Alvaro José Zamora Mata,2
672,227697,Garang Kuol,1
673,296254,Walid Cheddira,0
674,326758,Abolfazl Jalali,10


2. Total number of passes made under pressure, total number of completed passes made under pressure, and completion rate

In [4]:
passesDF['pass_outcome'] = passesDF['pass_outcome'].fillna(True)


# total number of passes played under pressure by player
pass_count = passesDF.groupby('player_id')['pass_outcome'].count().reset_index()
pass_count.rename(columns= {'pass_outcome': 'passes'}, inplace=True)

# Total of successful passes played under presures
succ_passes= passesDF.loc[~(passesDF.pass_outcome.isin(['Incomplete', 'Unknown', 'Pass Offside','Out', 'Injury Clearance']))]
succ_passes_count= succ_passes.groupby('player_id')['pass_outcome'].count().reset_index()
succ_passes_count.rename(columns= {'pass_outcome': 'passes_completed'}, inplace=True)

merged_passes = succ_passes_count.merge(pass_count, on='player_id', how='left')
merged_passes['completion_rate']= (merged_passes['passes_completed']/ merged_passes['passes']) * 100

In [5]:
merged_passes

,player_id,passes_completed,passes,completion_rate
0,2941,60,82,73.170732
1,2954,53,63,84.126984
2,2972,32,45,71.111111
3,2988,96,127,75.590551
4,2989,23,27,85.185185
...,...,...,...,...
668,156734,21,25,84.000000
669,184468,5,5,100.000000
670,227697,3,4,75.000000
671,296254,5,8,62.500000


In [6]:

# As of now let us import carries, passes, interceptions, pressurs, and dribbles data and extract the following metrics
# Pass compelition rate under pressure
# Successful passes made under pressure
# Pogressive passes 
# Progressive carries
# Interceptions
# Pressurs
# Succesfful dribbles 